In [84]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import votable, ascii
# import h5py
import json
import numpy as np
from types import SimpleNamespace

### SQL query on MAST

In [12]:
def z_cat_query(tl, br):
    string = "select * from catalogRecordRowStore where raMean>{:.2f} and raMean<{:.2f} and decMean>{:.2f} and decMean<{:.2f}".format(br[0], tl[0], br[1], tl[1])
    return string

In [3]:
def sexa_to_deci(c):
    sc = SkyCoord(c, unit=(u.hourangle, u.deg))
    return sc.ra.deg, sc.dec.deg

In [13]:
br = sexa_to_deci("00:03:19.1890576 +09:54:13.899849")
tl = sexa_to_deci("00:04:35.3301991 +10:12:15.172584")

In [7]:
print("test<{:.2f}".format(br[0]))

test<0.83


In [14]:
print(z_cat_query(tl, br))

select * from catalogRecordRowStore where raMean>0.83 and raMean<1.15 and decMean>9.90 and decMean<10.20


### Preparing prioritize.py input

In [96]:
# center = "00:03:52.6853271 +10:04:00.687612"
center = "00:03:48.7225972 +10:01:48.665376"
# guide = 1237678906782974123
# align = []
print(sexa_to_deci(center))

(0.9530108216666666, 10.030184826666668)


In [39]:
votb = (votable.parse_single_table(source='sdss8.xml')).to_table()

In [42]:
votb[votb['objID']==guide]['_RAJ2000', '_DEJ2000', 'rmag']

_RAJ2000,_DEJ2000,rmag
deg,deg,mag
float64,float64,float32
1.08345800,10.07273700,16.396


In [38]:
guide

1237678906782974123

In [45]:
archivez = ascii.read('archivez.csv')

In [52]:
list(archivez['RA'].data)

[0.93249,
 0.92447,
 0.94994,
 0.93804,
 0.95263,
 0.95326,
 0.96106,
 0.89773,
 0.89773,
 0.892,
 0.85752,
 0.95566,
 0.8243,
 1.09676,
 0.85188]

### Creating input file for prioritize.py

In [58]:
# RMJ 0003: USER MUST CHANGE THESE
maskRA = 0.96952219625
maskDEC = 10.06685767
zclust=0.37
slitPA=-2
BCGmag = 18.288
guideRA,guideDEC,guidemag = (001.08345800, +10.07273700, 16.396)
cosdec = np.cos(np.radians(guideDEC))
aligninfo = [[000.94828400, +09.96222900, 15.765],
            [001.05096900, +10.17006100, 16.073],
            [001.09680700, +10.13814800, 16.484],
            [001.06405100, +10.16007700, 18.052],
            [000.90873600, +09.97891300, 18.173],
            [001.06154100, +10.17188100, 19.690]]
specialtargets = []

In [88]:
json_data = {'maskRA': 0.96952219625,
'maskDEC': 10.06685767,
'zclust': 0.37,
'slitPA': -2,
'BCGmag': 18.288,
'guide': (001.08345800, +10.07273700, 16.396),
'cosdec': np.cos(np.radians(guideDEC)),
'aligninfo': [[000.94828400, +09.96222900, 15.765],
            [001.05096900, +10.17006100, 16.073],
            [001.09680700, +10.13814800, 16.484],
            [001.06405100, +10.16007700, 18.052],
            [000.90873600, +09.97891300, 18.173],
            [001.06154100, +10.17188100, 19.690]],
'specialtargets': []}

In [89]:
with open("file.json", 'w') as f:
    json.dump(json_data, f, indent=2)

In [91]:
with open("file.json", 'r') as f:
    json_data2 = json.load(f)

In [92]:
json_data2

{'maskRA': 0.96952219625,
 'maskDEC': 10.06685767,
 'zclust': 0.37,
 'slitPA': -2,
 'BCGmag': 18.288,
 'guide': [1.083458, 10.072737, 16.396],
 'cosdec': 0.9845865131128568,
 'aligninfo': [[0.948284, 9.962229, 15.765],
  [1.050969, 10.170061, 16.073],
  [1.096807, 10.138148, 16.484],
  [1.064051, 10.160077, 18.052],
  [0.908736, 9.978913, 18.173],
  [1.061541, 10.171881, 19.69]],
 'specialtargets': []}

In [93]:
v = SimpleNamespace(**json_data2)

In [86]:
v.maskRA

0.96952219625

In [94]:
v.guideRA,v.guideDEC,v.guidemag = v.guide

In [95]:
v.guideRA

1.083458

In [97]:
votb = (votable.parse_single_table(source='2/align.xml')).to_table()
# votb[votb['objID']==guide]['_RAJ2000', '_DEJ2000', 'rmag']

In [101]:
votb.columns

<TableColumns names=('_RAJ2000','_DEJ2000','_V','mode','q_mode','cl','SDSS8','m_SDSS8','Im','RA_ICRS','DE_ICRS','zsp','umag','e_umag','gmag','e_gmag','rmag','e_rmag','imag','e_imag','zmag','e_zmag','Q','ObsDate','objID')>

In [106]:
print(list(votb.iterrows('_RAJ2000','_DEJ2000','rmag')))

[(0.948284, 9.962229, 15.765), (1.001962, 9.969801, 19.387), (0.875299, 10.185863, 15.263), (0.846498, 10.140442, 15.871), (0.859229, 10.16519, 18.891)]


In [108]:
votb = (votable.parse_single_table(source="2/guide.xml")).to_table()


In [117]:
zipped = votb.iterrows('_RAJ2000','_DEJ2000','rmag')

In [119]:
dir(zipped)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [120]:
zip(*zipped)

In [121]:
v.guideRA,v.guideDEC,v.guidemag = list(votb.iterrows('_RAJ2000','_DEJ2000','rmag'))[0]

### Downloading the PS catalog

In [123]:
import ps_api

In [126]:
cl_pos = ps_api.resolve('RMJ000343.8+100123.8')

In [146]:
radius = (9 * u.arcmin).to(u.deg).value
# radius = .03

In [154]:
# constraints = {'nDetections.gt':1, 'rMeanKronMag.gt':-990}
constraints = {'rMeanKronMag.gt':-990}
constraints = {}

In [155]:
columns = """objID,raMean,decMean,rMeanKronMag""".split(',')
# strip blanks and weed out blank and commented-out values
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]

In [156]:
results = ps_api.ps1cone(*cl_pos, radius, columns=columns, **constraints)
# try including release="dr2" as an argument

In [157]:
with open('ps_api_res.csv', 'w') as f:
    f.write(results)